# 03 - Classes

Classes are a python structure which allow you to group data (attributes) and functions (methods) together in a single object. They are useful for providing a logical grouping of information and ways of interacting with that information. Classes are often used to represent abstract entities. For instance, a class could represent a person with attributes for name and date of birth plus a method to calculate their age based on their date of birth. 

### Class Syntax


### Example - Blank Class
Classes are defined with an `__init__()` method which sets the inital attributes you want to store in each object. The Car class below has nothing set in the `__init__()` and also has no methods so it is a completely blank class. A version of the class, called a class instance is initiated by calling the class and assigning it to a variable 

In [27]:
class Car:
    def __init__(self):
        pass

car = Car()

### Example - Initialisation with Attributes
We extend the previous example by defining atributes to initialise the Car object with. Similarly to functions we can provide default values and type hints. Note that as with functions, the type hints are not enforced at run time and merely act as a prompt to the developer. Note how each attribute is accessed from the class instance using the object.attribute syntax. 

In [28]:
class Car:
    def __init__(self, make: str = "Ford", model: str = "Mustang", year: int = 1960):
        self.make = make
        self.model = model
        self.year = year

car_with_defaults = Car()
car_with_custom_values = Car(make="Toyota", model="Corolla", year=2020)
car_contradicting_type_hints = Car(make=123, model=True, year="2021")

print("Car makes:")
print(car_with_defaults.make, car_with_custom_values.make, car_contradicting_type_hints.make)

print("Car models:")
print(car_with_defaults.model, car_with_custom_values.model, car_contradicting_type_hints.model)

print("Car years:")
print(car_with_defaults.year, car_with_custom_values.year, car_contradicting_type_hints.year)

Car makes:
Ford Toyota 123
Car models:
Mustang Corolla True
Car years:
1960 2020 2021


### Example - Initialisation with Attributes & Methods
We extend the previous example by defining methods which are effectively functions which act on the class. The first method `get_car_age` takes the current year as an argument and computes the age of the car. Note how within `get_car_age`, attributes of the class can be accessed via self.attribute. Methods such as these are called class methods and must have self as their first argument. Methods which do not reference any attributes or other methods of the class may omit the self argument. Such methods are called static methods and require the @staticmathod decorator. `count_wheels` is an example of a static method since it does not reference self anywhere in its definition. 

In [29]:
class Car:
    def __init__(self, make: str = "Ford", model: str = "Mustang", year: int = 1960):
        self.make = make
        self.model = model
        self.year = year

    def get_car_age(self, current_year: int) -> int:
        return current_year - self.year
    
    @staticmethod
    def count_wheels() -> int:
        return 4

car1 = Car()
car2 = Car(make="Toyota", model="Corolla", year=2020)

print("Car1 age:", car1.get_car_age(current_year=2025))
print("Car2 age:", car2.get_car_age(current_year=2025))

print("Car1 wheels:", car1.count_wheels())
print("Car2 wheels:", car2.count_wheels())


Car1 age: 65
Car2 age: 5
Car1 wheels: 4
Car2 wheels: 4


### Example - Getters, Setters & Deleters
Properties allow for a method to be treated like an attribute (i.e. the parenthesis are not needed when calling it). These are
called 'getters' and such methods require the @property decorator. Properties are especially useful when you want to use an attribute but want some additional logic, validation or calculation behind the scenes. In the example, `warranty_expiration` is a property so can be accessed as an attribute. But it is being computed on the fly, rather than being stored on the object. Note, in the example we assume each car has a warranty for 5 years. 

A setter method allows you to add some additional validation or processing before an attribute is set on a class instance. When a new class instance is defined, the logic in the year setter is run to validate whether the passed value is permissable. Note that it is not possible to define a setter without first defining a getter.

A deleter method allows you to run additional logic when a class attribute is deleted. In the example, when the year is removed, the missing_data attribute is set to True.  

In [30]:
from datetime import datetime

class Car:
    def __init__(self, make: str = "Ford", model: str = "Mustang", year: int = 1960):
        self.make = make
        self.model = model
        self.year = year

    @property
    def warranty_expiration(self):
        return self.year + 5

    @property
    def year(self) -> int:
        return self._year

    @year.setter
    def year(self, value: int):
        current_year = datetime.now().year
        if value > current_year:
            raise ValueError("Year cannot be in the future")
        elif value < 1886:
            raise ValueError("Year cannot be before the invention of the car")
        self._year = value  # store validated value
    
    @year.deleter
    def year(self):
        self.missing_data = True  # This will be set when year is deleted

car1 = Car()
print("Car1 warranty expiration:", car1.warranty_expiration)
print("Car1 year:", car1.year)

try:
    car2 = Car(make="Toyota", model="Corolla", year=1066)  # This will raise an error
except ValueError as e:
    print("Error:", e)

try:
    car3 = Car(make="Tesla", model="Model Alpha", year=2050)  # This will raise an error
except ValueError as e:
    print("Error:", e)

del car1.year  # This will delete the year but also the warranty expiration property due to the deleter method
print("Car1 Missing Data:", car1.missing_data)


Car1 warranty expiration: 1965
Car1 year: 1960
Error: Year cannot be before the invention of the car
Error: Year cannot be in the future
Car1 Missing Data: True


### Example - Inheritance
Classes can be based on another class, inheriting all attributes and methods but allowing for those attributes and method to be overwritten in the subclass. In the example below, the ElectricCar class inherits from the Car class. Note that inheritance works only in one direction. That is, instances of the ElectricCar class are necessarily members of the Car class, but instances of the Car class are not necessarily members of the ElectricCar class. The super() method, allows you to access attributes and methods of the parent class. In the example, super() is leveraged to initialise the subclass using the parent class' `__init__` and then the subclass is extended by adding the battery_capacity attribute. The functionality of methods from the parent class can be extended or modified by creating a method of the same name in the subclass. When this method is called on an instance of the subclass, the logic defined in the subclass is used rather than the logic from the parent class. Additionally, further methods which only exist in the subclass can be defined (see get_battery_range_km wich doesn't exist in the parent Car class)

In [31]:
class ElectricCar(Car):
    def __init__(self, make, model, year, battery_capacity_kwh):
        super().__init__(make, model, year)  # This sets the make, model and year as in the base Car class
        self.battery_capacity_kwh = battery_capacity_kwh  # We can also add additional attributes when we initialise a sub-class

    # We can override the functionality of methods in the base class by defining methods of the same name in the subclass
    @property
    def warranty_expiration(self):
        return self.year + 7
    
    def get_battery_range_km(self):
        range_km = self.battery_capacity_kwh * 5
        print(f"{self.battery_capacity} kWh battery provides a range of {range_km} km")
        return range_km
    

# Initialising objects of the classes
car = Car("Toyota", "Camry", 2023)
electric_car = ElectricCar("Tesla", "Model S", 2023, "100 kWh")

# One-way inheritance
print(isinstance(car, Car))                  # Output: True, car is an instance of Car
print(isinstance(car, ElectricCar))          # Output: False, car is not an instance of ElectricCar
print(isinstance(electric_car, Car))         # Output: True, electric_car is also an instance of Car since it inherits from it
print(isinstance(electric_car, ElectricCar)) # Output: True, electric_car is an instance of ElectricCar


print("Car warranty expiration:", car.warranty_expiration)
print("Electric Car warranty expiration:", electric_car.warranty_expiration)


True
False
True
True
Car warranty expiration: 2028
Electric Car warranty expiration: 2030
